<a href="https://colab.research.google.com/github/mannmoshe/text-recognition/blob/main/text_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# based on:
# https://deeplearningcourses.com/c/data-science-natural-language-processing-in-python 
# https://www.udemy.com/data-science-natural-language-processing-in-python Section 7 Cipher Decryption

# Author: Moshe Mann

import numpy as np
np.seterr(divide = 'ignore') 

import string
import random
import requests

In [2]:
for l in range(1488, 1515):
  print(l - 1488, chr(l))

0 א
1 ב
2 ג
3 ד
4 ה
5 ו
6 ז
7 ח
8 ט
9 י
10 ך
11 כ
12 ל
13 ם
14 מ
15 ן
16 נ
17 ס
18 ע
19 ף
20 פ
21 ץ
22 צ
23 ק
24 ר
25 ש
26 ת


In [3]:
### the language model

# initialize Markov matrix
# MM: matrix for the bi-gram probabilites
M = np.ones((27, 27))

# initial state distribution
# MM: vector for the uni-gram probabilites
# MM: uni-gram probabilites calculated only by first letters of the words
pi = np.zeros(27)

# a function to update the Markov matrix
def update_transition(ch1, ch2):
  # ord('א') = 1488, ord('ב') = 1489, ...
  i = ord(ch1) - 1488 # 'א' will mapped to 0, 'ב' will mapped to 1 and so on
  j = ord(ch2) - 1488
  M[i,j] += 1

# a function to update the initial state distribution
def update_pi(ch):
  i = ord(ch) - 1488
  pi[i] += 1

# get the log-probability of a word / token
def get_word_prob(word):
  # print("word:", word)
  i = ord(word[0]) - 1488
  logp = np.log(pi[i])

  for ch in word[1:]:
    j = ord(ch) - 1488
    logp += np.log(M[i, j]) # update prob
    i = j # update i to j

  return logp

# get the probability of a sequence of words
def get_sequence_prob(words):
  # if input is a string, split into an array of tokens
  if type(words) == str:
    words = words.split()

  logp = 0
  for word in words:
    logp += get_word_prob(word)
  return logp

In [4]:
# with open('torah_text.txt', encoding='ISO-8859-8') as f:
#     torah_text = f.read()
req = requests.get("https://raw.githubusercontent.com/mannmoshe/text-recognition/main/torah_text.txt")
req.encoding = 'ISO-8859-8'
torah_text = req.text

In [5]:
torah_text[:50]

'  בראשית ברא אלהים את השמים ואת הארץ והארץ היתה תה'

מספר הפעמים שהאות א מופיעה שלא בסוף המילה

In [6]:
torah_text.count('א') - torah_text.count('א ')

23502

In [7]:
torah_text.count('אב')

972

In [8]:
tokens = torah_text.split()

for token in tokens:
  # update the model

  # first letter
  ch0 = token[0]
  update_pi(ch0)

  # other letters
  for ch1 in token[1:]:
    update_transition(ch0, ch1)
    ch0 = ch1

pi_initial_count = pi.copy()
M_initial_count = M.copy()

# normalize the probabilities
pi /= pi.sum()
M /= M.sum(axis=1, keepdims=True)

first letter distribution:

In [9]:
pi_initial_count

array([13177.,  6162.,   532.,   549.,  8724., 13776.,   634.,  1073.,
         251.,  5585.,     0.,  3434.,  6859.,     0.,  5383.,     0.,
        1442.,   291.,  3983.,     0.,  1022.,     0.,   515.,   793.,
         857.,  2736.,  2133.])

The probabilties vector of first letters in word:

In [10]:
pi

array([0.16489595, 0.07711079, 0.00665741, 0.00687014, 0.10917145,
       0.17239179, 0.00793383, 0.01342744, 0.00314099, 0.06989025,
       0.        , 0.04297281, 0.08583299, 0.        , 0.06736244,
       0.        , 0.01804508, 0.00364155, 0.04984295, 0.        ,
       0.01278923, 0.        , 0.00644467, 0.00992354, 0.01072443,
       0.03423809, 0.0266922 ])

In [11]:
pi.sum()

1.0

In [12]:
M_initial_count[0]

array([2.000e+00, 9.730e+02, 1.600e+01, 5.360e+02, 9.610e+02, 1.111e+03,
       1.030e+02, 1.098e+03, 4.000e+00, 1.073e+03, 1.010e+02, 4.900e+02,
       4.090e+03, 3.880e+02, 1.688e+03, 1.060e+02, 5.920e+02, 1.140e+02,
       5.500e+01, 5.300e+01, 1.450e+02, 2.000e+00, 5.600e+01, 3.200e+01,
       1.460e+03, 2.737e+03, 5.543e+03])

In [13]:
M_initial_count[0].sum()

23529.0

The probabilties matrix of bi-gram where the first letter is: א

In [14]:
M[0]

array([8.50014875e-05, 4.13532237e-02, 6.80011900e-04, 2.27803987e-02,
       4.08432148e-02, 4.72183263e-02, 4.37757661e-03, 4.66658167e-02,
       1.70002975e-04, 4.56032981e-02, 4.29257512e-03, 2.08253644e-02,
       1.73828042e-01, 1.64902886e-02, 7.17412555e-02, 4.50507884e-03,
       2.51604403e-02, 4.84508479e-03, 2.33754091e-03, 2.25253942e-03,
       6.16260785e-03, 8.50014875e-05, 2.38004165e-03, 1.36002380e-03,
       6.20510859e-02, 1.16324536e-01, 2.35581623e-01])

In [15]:
M.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [16]:
get_sequence_prob('וידבר יהוה אל משה לאמר')

-38.291733346990895

In [17]:
letters = [chr(l) for l in range(1488, 1515)]

random_texts = []
random_scores = []
for i in range(1000):
  random_text = ''.join(random.choice(letters) for i in range(5)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(4)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(2)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(3)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(4)) 
  #print (random_text)
  random_scores.append(get_sequence_prob(random_text))
  random_texts.append(random_text)

print(len(random_scores))
print(random_texts[:10])
print(random_scores[:10])
print('average_score:', sum(random_scores) / len(random_scores))
print("\n")
fixed_random_scores = [value for value in random_scores if value != float('-inf')]
print(len(fixed_random_scores))
#print(fixed_random_scores)
print('average_score:', sum(fixed_random_scores) / len(fixed_random_scores))

1000
['םשעלן צוףר סב אמפ ורבח', 'אשיזל שהכק הל תסס ףגזפ', 'יןקפס ךךקר ץז קלו ךצשנ', 'גךחרר םזתמ סם אזש ערטל', 'חםםכר ןךךן תז הכט תכסצ', 'קםךףב שדטך צר גםס לאדץ', 'ודכחע יקרע דב פפל דטךצ', 'זאטנא דשטא צנ יאז רלזח', 'םמטפץ אעאי ףת יסב םגטנ', 'םוטטא טשכת ץכ תשא נצזט']
[-inf, -inf, -inf, -inf, -inf, -74.09213020184623, -80.50798956019156, -80.74025092894857, -inf, -inf]
average_score: -inf


361
average_score: -73.70269178371612


get_sequence_prob('וידבר יהוה אל משה לאמר') is bigger than average score of a sentence with a same structure.

In [18]:
get_sequence_prob('ואהבתם את הגר')

-27.056798620938007

In [19]:
get_sequence_prob('ואהבת לרעך כמוך')

-42.81038222930017

In [20]:
get_sequence_prob('וילך משה ויאמר אל עמו הנה פרעה משלח אתכם ואת טפכם')

-101.2397427430866

In [21]:
letters = [chr(l) for l in range(1488, 1515)]

random_texts = []
random_scores = []
for i in range(1000):
  random_text = ''.join(random.choice(letters) for i in range(4)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(3)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(5)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(2)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(3)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(3)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(4)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(4)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(4)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(3)) + ' ' \
                 + ''.join(random.choice(letters) for i in range(4)) 
  #print (random_text)
  random_scores.append(get_sequence_prob(random_text))
  random_texts.append(random_text)

print(len(random_scores))
print(random_texts[:10])
print(random_scores[:10])
print('average_score:', sum(random_scores) / len(random_scores))
print("\n")
fixed_random_scores = [value for value in random_scores if value != float('-inf')]
print(len(fixed_random_scores))
#print(fixed_random_scores)
print('average_score:', sum(fixed_random_scores) / len(fixed_random_scores))

1000
['נתהד ברק רךןמם טכ לוצ כטר מפמס אםתב שךכב עטב גםרל', 'מגזל ןץך גטסוך או ךחג ץחט זאדף פכךח ףלאכ חןצ צצלס', 'ךנשה סזא ממעפש דע ודך ץונ אשסר יבוא טטםז חנס רטסן', 'דצדב כהז שתץרת וו חור חצז חףפן יטזת כואפ הטנ ךיןט', 'דתהצ ןשא זאסהו נח מהן קרו גשטע גסתט כןחכ רסש ןדגק', 'לעחע שהכ ירזקכ רף ץזק כעכ וצגת בןכו ללאף הםא דאשב', 'דיגה סהו יסץנב נג ךנל לדו בלףה מדום נמיש בכו סךטו', 'בגכד ודת ץץצצט אן ךגע עבע מעטם חיטף ועהה יבך ףטמת', 'טדץן צץג שהונץ לא חספ גכז צוכא סילש םמגת פהר ףץףק', 'ניףן זסן םסטףד סל רחט תיד ךסתת תתחך ףחעפ ידא דצהם']
[-153.03307780575938, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]
average_score: -inf


101
average_score: -161.56389441493928


Recognize true text 5 characters long in skips of 2.

In [22]:
torah_text[:50]

'  בראשית ברא אלהים את השמים ואת הארץ והארץ היתה תה'

In [68]:
torah_text_no_spaces = torah_text.strip().replace(" ", "").replace("  ", "")

In [79]:
print(torah_text_no_spaces[:50]+'\n')
print(torah_text_no_spaces[-50:]+'\n')
print(len(torah_text_no_spaces))

בראשיתבראאלהיםאתהשמיםואתהארץוהארץהיתהתהוובהווחשךעל

רצוולכלהידהחזקהולכלהמוראהגדולאשרעשהמשהלעיניכלישראל

304600


In [80]:
torah_skips = ''
for i in range(0, len(torah_text_no_spaces), 2):
  torah_skips += torah_text_no_spaces[i]

In [81]:
torah_skips[:100]

'באיבאליאהמםאהרואץיההוהושעפיהםרחלימחתלנהיוארליייווהאריאליאהוכטבידאהםיהוויהשוקאלילויםלשקאיהיירוהברואדי'

In [82]:
words_probs_20 = {}
for i in range(len(torah_skips) - 20):
  word = torah_skips[i:i + 20]
  if get_sequence_prob(word) != float('-inf'):
    words_probs_20[word] = get_sequence_prob(word)
words_probs_20 = dict(sorted(words_probs_20.items(), key=lambda item: item[1], reverse=True))
words_probs_20

{'ליהמויההויהיהאלויהיא': -44.08113883474636,
 'המויההויהיהאלויהיארא': -44.60101107602181,
 'יהמויההויהיהאלויהיאר': -45.02072835211838,
 'ליאתויוהולקדבישאליאת': -45.41074255019094,
 'אהויותתיויעלארבאלניו': -46.75835038580793,
 'הדליהמויההויהיהאלויה': -46.88343005717402,
 'אתויוהולקדבישאליאתמן': -46.95681901063987,
 'יאתויוהולקדבישאליאתמ': -47.29372863386202,
 'דליהמויההויהיהאלויהי': -47.31267809831628,
 'אשחליאתויוהולקדבישאל': -47.403494586730666,
 'ויהעואלארהלכימויהאמש': -47.73754153602982,
 'היהעומאתויאשאשהישביו': -47.95043392257805,
 'עריאשחליאתויוהולקדבי': -48.011824427222166,
 'והמויםטראתרהרתיביואמ': -48.01728703546937,
 'חליאתויוהולקדבישאליא': -48.32903491833571,
 'היניאתובהואהיומיואהא': -48.57157022168886,
 'ודברארומהלהאשרהאיהוי': -48.74896930871311,
 'שחליאתויוהולקדבישאלי': -48.86978417549577,
 'הרויהעואלארהלכימויהא': -48.912732057228894,
 'יעריאשחליאתויוהולקדב': -49.009835602960315,
 'בהיהעומאתויאשאשהישבי': -49.04693093782239,
 'יהעומאתויאשאשהישביות': -49.08593081323984,
 'ותוי

In [83]:
words_probs_15 = {}
for i in range(len(torah_skips) - 15):
  word = torah_skips[i:i + 15]
  if get_sequence_prob(word) != float('-inf'):
    words_probs_15[word] = get_sequence_prob(word)
words_probs_15 = dict(sorted(words_probs_15.items(), key=lambda item: item[1], reverse=True))
words_probs_15

{'ויההויהיהאלויהי': -31.04310673601242,
 'הויהיהאלויהיארא': -31.36510829645146,
 'מויההויהיהאלויה': -32.72746904447306,
 'ליהמויההויהיהאל': -32.99753544232238,
 'המויההויהיהאלוי': -33.13815426590379,
 'יההויהיהאלויהיא': -33.21591068383969,
 'אתויוהולקדבישאל': -33.3460027137177,
 'ליאתויוהולקדביש': -33.42425819402061,
 'ההויהיהאלויהיאר': -33.66457267992939,
 'יהמויההויהיהאלו': -33.801869556974765,
 'ויותתיויעלארבאל': -34.15652554227262,
 'אתויאשאשהישביות': -34.30320275468704,
 'ואלואראיאהראתוא': -34.31574844575897,
 'הויותתיויעלארבא': -34.57740474006123,
 'ליהיויהלהיאלאומ': -34.614025731008546,
 'ויוהשביההוהיואי': -34.68973749661447,
 'אויוהשביההוהיוא': -34.69939051172779,
 'בליהיויהלהיאלאו': -34.73062026793942,
 'אהויותתיויעלארב': -34.770455927937626,
 'ולהיוידיתואיואר': -34.83775255687483,
 'ויוהולקדבישאליא': -34.869220629748526,
 'הותותהאראראיתוי': -35.00480482012214,
 'ויוישיוהשישהנימ': -35.01043005057966,
 'אלואראיאהראתואר': -35.12102531620937,
 'ויהיםדילליםניוא': -35.1238789932650

In [86]:
words_probs_10 = {}
for i in range(len(torah_skips) - 10):
  word = torah_skips[i:i + 10]
  if get_sequence_prob(word) != float('-inf'):
    words_probs_10[word] = [get_sequence_prob(word), i, i+10]
words_probs_10 = dict(sorted(words_probs_10.items(), key=lambda item: item[1], reverse=True))
words_probs_10

{'ויהיהאלויה': [-19.61682541097137, 78731, 78741],
 'הויהיהאלוי': [-19.90225148633344, 78730, 78740],
 'יהיהאלויהי': [-20.80136822611412, 78732, 78742],
 'היויאלהיוי': [-20.831798218967442, 15978, 15988],
 'ויההויהיהא': [-21.147195023425287, 78727, 78737],
 'ליאתויוהול': [-21.275324576872816, 94102, 94112],
 'וימהוהותים': [-21.362314943029144, 43454, 43464],
 'אתויוהולקד': [-21.381304653172325, 94104, 94114],
 'היהאלויהיא': [-21.407613367706208, 78733, 78743],
 'עתואתוהולא': [-21.52264144880855, 2110, 2120],
 'ליהיויהלהי': [-21.551369622167815, 15772, 15782],
 'והוישהאשהע': [-21.586166632294916, 14170, 14180],
 'והמויםטראת': [-21.742288534076994, 82086, 82096],
 'ויוהולקדבי': [-21.794468570097433, 94106, 94116],
 'ויויארויוא': [-21.89469563603752, 141090, 141100],
 'אשריואראדי': [-21.975030414306605, 14409, 14419],
 'ויוהמרהולי': [-21.975753747680194, 144982, 144992],
 'ויותניותהי': [-21.98669381640897, 6010, 6020],
 'אתהאמאראתה': [-22.026124333152566, 7532, 7542],
 'אתויאשאשהי': [-22.